# Data Partitioning

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
random_state = 123

In [51]:
print("Start Data Partitioning")

Start Data Partitioning


In [52]:
# read cleaned data
df = pd.read_csv(r'../raw_data/label_data.csv')
df.head()

,Patient_ID,Date_MRI,ID_MRI_Machine,Entry_date,Operation_date,Adenoma_size,Diagnosis,Category,Patient_age,Prolactin,...,Pre_OP_hormone_intakt,Post_OP_hormone_cortico,Post_OP_hormone_gonado,Post_OP_hormone_somato,Post_OP_hormone_ADH,Post_OP_hormone_thyreo,Post_OP_hormone_hyperprolaktin,Post_OP_hormone_keine,Post_OP_hormone_intakt,Label_Quality
0,300146159,2023-05-11 09:00:00,MRI3,2021-09-01,2021-09-17,makro,inaktiv (gonado),non-prolaktinom,57,0.0,...,0,0,1,0,0,0,0,0,0,NaN
1,762512,2023-05-06 08:12:00,MRI3,2018-09-01,2018-09-19,makro,gh,non-prolaktinom,66,0.0,...,0,0,0,0,0,1,0,0,0,NaN
2,365189,2023-05-05 14:19:00,MRI3,NaN,NaN,NaN,NaN,NaN,32,0.0,...,0,0,0,0,0,0,0,0,0,keine daten
3,543641,2023-05-05 07:54:00,MRI2,2006-01-01,2009-06-04,mikro,acth,non-prolaktinom,39,0.0,...,0,0,0,0,0,0,0,1,0,NaN
4,300302329,2023-05-02 12:04:00,MRI3,2021-10-01,2022-01-12,makro,inaktiv,non-prolaktinom,56,0.0,...,1,0,0,0,0,0,0,0,1,NaN


## Train/Test Split Strategy
We will Split the Data into a Train- and Testset. 
We are splitting each patient fully into either the train or the test set to avoid data leakage.
This is ensured checking for Patient ID duplicates (assert statement beneath) in the Dataframe. If each row only corresponds to one patient we can savely split the dataframe.


Also we are gonna stratify the split on the binary labels to ensure that the train- and testset include about the same of each class.

In [53]:
# drop patients without a label
df= df[~df["Category"].isna()]

In [54]:
# Patient ID Duplicate Check
assert len(df[df["Patient_ID"].duplicated()]) == 0

In [55]:
# split columns into features and labels
X = df.drop(columns=["Category"])
y = df["Category"]
# stratified train/test split on labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

In [56]:
print("Total Dataframe Train rows:", len(X_train))
print("Total Dataframe Test rows:", len(X_test))

Total Dataframe Train rows: 138
Total Dataframe Test rows: 35


In [57]:
label_diff = ((y_train.value_counts(normalize=True) - y_test.value_counts(normalize=True)) *100).iloc[1]

In [58]:
print(f"Label Distribution relative Difference between Train- and Testset:\n","±",np.round(label_diff,3),"%")

Label Distribution relative Difference between Train- and Testset:
 ± 0.166 %


In [59]:
# Create dataframes for training and test data
train_data = pd.DataFrame(X_train)
train_data['Category'] = y_train

test_data = pd.DataFrame(X_test)
test_data['Category'] = y_test

In [60]:
# read cleaned labor data
labor_data = pd.read_csv(r'../raw_data/labor_data_preprocessed.csv',dtype={"Fall Nr.": str}).rename(columns={"Fall Nr.": "Case_ID","PATIENT_NR": "Patient_ID",})
labor_data.head()

,Patient_ID,Case_ID,COR60,FSH,FT4,LH,PROL,TEST
0,4858,0041324884,NaN,48.0,144.0,20.0,525.0,218.0
1,6144,0040619017,NaN,NaN,NaN,NaN,NaN,876.0
2,7375,0041172058,NaN,103.0,123.0,520.0,660.0,117.0
3,18851,0040954960,NaN,NaN,103.0,180.0,NaN,NaN
4,18851,0041276535,NaN,131.0,120.0,129.0,657.0,NaN


In [61]:
df_more_data = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='w duplicates')
df_more_data =df_more_data[["Fall Nr.","PID"]]
df_more_data= df_more_data.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",})
df_more_data

,Case_ID,Patient_ID
0,0041835743,300146159
1,0041708812,762512
2,0041892695,365189
3,0041725372,543641
4,0041843364,300302329
...,...,...
1193,0004213315,112374
1194,0004211936,153807
1195,0004180070,719666
1196,0004139115,313269


In [62]:
train_data_merged = train_data.merge(df_more_data,how='right',on='Patient_ID')
train_data_merged = train_data_merged.merge(labor_data,how='left',on=['Case_ID',"Patient_ID"])
test_data_merged = test_data.merge(df_more_data,how='right',on='Patient_ID')
test_data_merged = test_data_merged.merge(labor_data,how='left',on=['Case_ID',"Patient_ID"])

In [65]:
# Save the training and test data to CSV files
train_data_merged.to_csv(r'../data/train_data.csv', index=False)
test_data_merged.to_csv(r'../data/test_data.csv', index=False)

In [66]:
assert len(train_data_merged.Patient_ID.unique()) == len(train_data.Patient_ID.unique())
assert len(test_data_merged.Patient_ID.unique()) == len(test_data.Patient_ID.unique())

AssertionError: 

In [ ]:
print("End Data Partitioning")

End Data Partitioning
